In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Downloading network

In [2]:
USOS_ZONA_7 = gpd.read_file("USOS_ZONA_7.shp")

In [3]:
schema = "denue"
table = "denue_2022"

entidad = "cve_ent"
localidad = "cve_loc"
manzana = "manzana"
municipio = "cve_mun"
# id = "id"
latitud = "latitud"
longitud = "longitud"
codigo_act = "codigo_act"
ageb = "ageb"
per_ocu = "per_ocu"

query_censo = f"""
SELECT 
  "{entidad}",
  "{localidad}",
  "{manzana}",
  "{municipio}",
  "{latitud}",
  "{longitud}",
  "{codigo_act}",
  "{ageb}",
  "{per_ocu}",
  "geometry"
FROM {schema}.{table}"""

Denue = aup.gdf_from_query(query_censo, geometry_col='geometry')
print(Denue.shape)
Denue.head(2)

(5528698, 10)


,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry
0,01,0001,056,001,21.901488,-102.289962,436112,198A,0 a 5 personas,POINT (-102.28996 21.90149)
1,01,0041,030,005,21.992029,-102.288802,436112,0463,0 a 5 personas,POINT (-102.28880 21.99203)


## Clasificar las distintas actividades económicas con base en su terminación en "codigo_act" basándonos en el Directorio Estadístico Nacional de Unidades Económicas

In [4]:
# Función para clasificar según la terminación de codigo_act
def asignar_tipo(codigo):
    if pd.isna(codigo):  # Si está vacío
        return 'Sin código'
    
    # Asegurar que sea string para evaluar el código
    codigo_str = str(codigo).strip()

    if not codigo_str.isdigit():
        return 'Código inválido'
    
    # Define tus conjuntos de códigos
    industria = {'111', '112', '113', '114', '115', '211', '212', '236', '237', '238',
                 '311', '312', '313', '314', '315', '316', '317', '318', '319', '320',
                 '321', '322', '323', '324', '325', '326', '327', '328', '329', '330',
                 '331', '332', '333', '334', '335', '336', '337', '338', '339', '551'}
    
    servicios = {'213', '221', '222', '481', '482', '483', '484', '485', '486',
                 '487', '488', '489', '490', '491', '492', '493', '521', '522', '523',
                 '524', '525', '531', '532', '533', '541', '561', '562', '721', '722',
                 '811', '812', '813', '814'}
    
    comercio = {'431', '432', '433', '434', '435', '436', '437', '461', '462',
                '463', '464', '465', '466', '467'}
    
    cultural_recreativo = {'511', '512', '513', '514', '515', '516', '517', '518', '519',
                           '711', '712', '713'}
    
    educacion = {'611'}
    
    salud = {'621', '622', '623', '624'}
    
    # Verificar si el código está en alguno de los conjuntos
    if codigo_str[-3:] in industria:
        return 'Industria'
    elif codigo_str[-3:] in servicios:
        return 'Servicios'
    elif codigo_str[-3:] in comercio:
        return 'Comercio'
    elif codigo_str[-3:] in cultural_recreativo:
        return 'Cultural/Recreativo'
    elif codigo_str[-3:] in educacion:
        return 'Educación'
    elif codigo_str[-3:] in salud:
        return 'Salud'
    else:
        return 'Desconocido'

# Aplica la función al DataFrame
Denue['tipo_act'] = Denue['codigo_act'].apply(asignar_tipo)

In [5]:
Denue.head(3)

,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry,tipo_act
0,01,0001,056,001,21.901488,-102.289962,436112,198A,0 a 5 personas,POINT (-102.28996 21.90149),Industria
1,01,0041,030,005,21.992029,-102.288802,436112,0463,0 a 5 personas,POINT (-102.28880 21.99203),Industria
2,01,0001,015,001,21.851988,-102.319345,433312,1439,0 a 5 personas,POINT (-102.31934 21.85199),Industria


In [20]:
Denue.shape

(5528698, 11)

In [22]:
Denue.columns

Index(['cve_ent', 'cve_loc', 'manzana', 'cve_mun', 'id', 'latitud', 'longitud',
       'codigo_act', 'ageb', 'geometry', 'tipo_act'],
      dtype='object')

In [13]:
Denue["tipo_act"].unique()

array(['Industria', 'Desconocido', 'Servicios', 'Cultural/Recreativo',
       'Educación', 'Comercio', 'Salud'], dtype=object)

In [17]:
desconocidos = Denue[Denue['tipo_act'] == 'Desconocido']

# Mostrar los valores únicos de codigo_act que no clasificaron
desconocidos_codigos = desconocidos['codigo_act'].unique()

print("Códigos que quedaron como 'Desconocido':")
print(desconocidos_codigos)

Códigos que quedaron como 'Desconocido':
['435220' '435210' '434225' '434226' '433510' '434223' '431130' '435110'
 '435419' '431199' '434229' '434219' '432119' '432130' '431160' '431192'
 '431122' '431110' '431140' '433220' '435411' '431121' '434224' '433110'
 '433210' '461110' '431150' '435412' '431180' '461130' '321910' '332310'
 '321920' '337120' '315223' '315999' '311110' '311820' '315224' '313230'
 '311993' '315229' '311520' '316991' '311910' '311614' '316999' '311422'
 '311613' '311421' '311423' '312149' '311230' '311340' '311999' '314120'
 '314912' '314991' '431193' '431170' '431191' '434240' '432120' '433410'
 '434227' '433430' '321999' '315225' '315192' '321992' '315210' '315110'
 '434230' '431220' '332710' '339999' '333999' '336390' '337110' '333610'
 '339930' '331510' '339950' '333411' '333510' '336370' '336310' '339940'
 '331220' '332110' '339913' '336991' '337210' '323119' '326191' '325510'
 '325620' '326199' '325610' '326150' '322299' '326130' '323120' '326220'
 '327215' 

In [18]:
# Contar cuántas veces aparece cada código no reconocido
desconocidos['codigo_act'].value_counts()

codigo_act
461110    604289
812110    223088
311830    110131
461130     94157
813210     86481
           ...  
212292         2
339911         2
486110         1
486990         1
212397         1
Name: count, Length: 645, dtype: int64

In [21]:
# Total de registros marcados como 'Desconocido'
total_desconocidos = (Denue['tipo_act'] == 'Desconocido').sum()

print(f"Total de 'Desconocido': {total_desconocidos}")

Total de 'Desconocido': 2999365


### Relación espacio -> espacio

In [23]:
# Relación manzana -> ageb
manzana_por_ageb = Denue[['manzana', 'ageb']].drop_duplicates().sort_values(by=['ageb', 'manzana'])

# ¿Cuántas manzanas por ageb?
conteo_manzanas_por_ageb = manzana_por_ageb.groupby('ageb').size().reset_index(name='total_manzanas')

conteo_manzanas_por_ageb.head()

,ageb,total_manzanas
0,0010,69
1,0011,80
2,0012,63
3,0013,85
4,0014,83


In [26]:
conteo_manzanas_por_ageb.isna().sum()

ageb              0
total_manzanas    0
dtype: int64

In [24]:
# Relación ageb -> localidad
ageb_por_localidad = Denue[['ageb', 'cve_loc']].drop_duplicates().sort_values(by=['cve_loc', 'ageb'])

# ¿Cuántos agebs por localidad?
conteo_agebs_por_localidad = ageb_por_localidad.groupby('cve_loc').size().reset_index(name='total_agebs')

conteo_agebs_por_localidad.head()

,cve_loc,total_agebs
0,0001,7375
1,0002,333
2,0003,373
3,0004,245
4,0005,301


In [27]:
conteo_agebs_por_localidad.isna().sum()

cve_loc        0
total_agebs    0
dtype: int64

In [25]:
# Relación localidad -> municipio
localidad_por_municipio = Denue[['cve_loc', 'cve_mun']].drop_duplicates().sort_values(by=['cve_mun', 'cve_loc'])

# ¿Cuántas localidades por municipio?
conteo_localidades_por_municipio = localidad_por_municipio.groupby('cve_mun').size().reset_index(name='total_localidades')

conteo_localidades_por_municipio.head()

,cve_mun,total_localidades
0,001,609
1,002,385
2,003,425
3,004,435
4,005,333


In [28]:
conteo_localidades_por_municipio.isna().sum()

cve_mun              0
total_localidades    0
dtype: int64

In [29]:
# Relación municipio -> estado
municipio_por_estado = Denue[['cve_mun', 'cve_ent']].drop_duplicates().sort_values(by=['cve_ent', 'cve_mun'])

# ¿Cuántos municipios por estado?
conteo_municipios_por_estado = municipio_por_estado.groupby('cve_ent').size().reset_index(name='total_municipios')

conteo_municipios_por_estado.head()

,cve_ent,total_municipios
0,01,11
1,02,6
2,03,5
3,04,12
4,05,38


In [30]:
conteo_municipios_por_estado.isna().sum()

cve_ent             0
total_municipios    0
dtype: int64

#### Mostrar de manera completa las relaciones y jerarquías entre los distintos espacios

In [6]:
jerarquia_completa = Denue[['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana']].drop_duplicates().sort_values(by=['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana'])

jerarquia_completa.head(10)

,cve_ent,cve_mun,cve_loc,ageb,manzana
5644,01,001,0001,006A,003
8878,01,001,0001,006A,009
88,01,001,0001,0106,009
3760,01,001,0001,0106,011
9766,01,001,0001,0182,031
2381,01,001,0001,0229,001
817,01,001,0001,0229,002
11524,01,001,0001,0229,003
1532,01,001,0001,0229,004
45897,01,001,0001,0229,007


In [7]:
conteo_manzanas = jerarquia_completa.groupby(['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana']) \
    .size().reset_index(name='valor') # Contar cuántas manzanas hay por agrupación

In [8]:
conteo_manzanas

,cve_ent,cve_mun,cve_loc,ageb,manzana,valor
0,01,001,0001,006A,003,1
1,01,001,0001,006A,009,1
2,01,001,0001,0106,009,1
3,01,001,0001,0106,011,1
4,01,001,0001,0182,031,1
...,...,...,...,...,...,...
975144,32,058,0001,0123,007,1
975145,32,058,0001,0123,009,1
975146,32,058,0001,0123,013,1
975147,32,058,0001,0123,020,1


## Crear centroides

In [9]:
print(Denue.crs)

epsg:4326


In [ ]:
jerarquia_completa = jerarquia_completa.merge(
    Denue[['manzana', 'geometry', 'per_ocu']],
    on='manzana', 
    how='left'
)

# Convertir en GeoDataFrame
jerarquia_completa = gpd.GeoDataFrame(jerarquia_completa, geometry='geometry')

jerarquia_completa.set_crs(epsg=4326, inplace=True)

In [12]:
# Reproyectar
jerarquia_completa.set_crs = 32614

# Calcular el centroide
jerarquia_completa['centroide'] = jerarquia_completa['geometry'].centroid

# Generar un buffer de 500 metros
jerarquia_completa['buffer_500m'] = jerarquia_completa['centroide'].buffer(500)

# Visualizar en el sistema original EPSG:4326 para mapas web
jerarquia_completa_buffers = jerarquia_completa.set_geometry('buffer_500m')
jerarquia_completa_buffers = jerarquia_completa_buffers.to_crs(epsg=4326)

KeyError: 'geometry'

In [ ]:
# Visualiza en geopandas
jerarquia_completa_buffers.plot()